In [2]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv(r"*insertpath\newsdata.csv")
df_time = pd.read_csv(r"*insertpart\stockprice.csv")

In [15]:
df["source"] = df["source"].map(lambda source: eval(source)["id"])

## Round off article publication time to nearest 5.00 mins

In [16]:
df["timestamp"] = df["publishedAt"].str.split("T")

In [17]:
time_boundary = []

for i in range(0,61,5):
    time_boundary.append(i)

def timeformat(time, timedelay):
    
    hour = int(time[0:2]) - 5 #UTC to NASDAQ time
    minute = int(time[3:5]) + timedelay
    
    if minute > 60:
        hour += 1
        minute = minute - 60
    
    ##if minute between 55 and 60, checks how close it is to each and adjusts accordingly
    for i in range(0, len(time_boundary) - 1):
        if time_boundary[i] - minute < 0 | time_boundary[i+1] - minute >= 0:
            avgtime = (1/2)*(time_boundary[i] + time_boundary[i+1])
            
            if avgtime - minute >= 0: 
                minute = time_boundary[i]
            elif avgtime - minute < 0:
                if time_boundary[i+1] == 60:
                    minute = 0
                    hour += 1
                else:
                    minute = time_boundary[i+1]
            else: 
                print("error")
            break
            
    if hour < 10:
        if hour < 0:
            hour = str(24 + hour) #date discrepancy is acknowledged
        else:
            hour = "0" + str(hour)
    else: 
        hour = str(hour)
        
    if minute == 0 or minute == 5:
        minute = "0" + str(minute)
    else:
        
        minute = str(minute)
        
    time = str(hour)+":"+str(minute)+":"+"00"
    
    return(time)

In [18]:
df["date"] = df["timestamp"].apply(lambda timestamp: timestamp[0])
df["time"] = df["timestamp"].apply(lambda time: time[1][0:8])

In [19]:
df["time_before"] = df["date"] + " " + df["time"].apply(lambda x: timeformat(time=x, timedelay=0))
df["time_after"] = df["date"] + " " + df["time"].apply(lambda x: timeformat(time=x, timedelay=20))

In [20]:
df = df.drop(["publishedAt", "date", "time", "timestamp"], axis=1)

## Merge df_time with df

In [22]:
df = df.merge(df_time, how="left", left_on="time_before", right_on="time")
df = df.merge(df_time, how="left", left_on="time_after", right_on="time")
df = df.drop_duplicates(subset="content").reset_index(drop=True)

In [23]:
df.rename({"opening prices_x": "price_before", "opening prices_y": "price_after"}, axis=1, inplace = True)

df.drop(["time_x", "time_y"],axis=1, inplace = True)
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)

In [28]:
df.count()

content         186
source          186
title           186
time_before     186
time_after      186
price_before    186
price_after     186
dtype: int64

In [29]:
df["price_change"] = df["price_after"] - df["price_before"]

In [30]:
df["price_movement"] = df["price_change"].map(lambda x: int(x))
df["price_movement"][df["price_change"] > 0] = 1
df["price_movement"][df["price_change"] < 0] = 0

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
def concat_LEARN_MORE(content):
    if content[len(content)-10:] == "LEARN MORE":
        return content[0:len(content)-10]
    else: return content

In [32]:
df["content"] = df["content"].apply(concat_LEARN_MORE)

In [34]:
df.to_csv(r"*insertpath\Data.csv", index = False)

In [7]:
df.sample(5)

,content,source,title,time_before,time_after,price_before,price_after,price_change,price_movement
21,"When you're watching a car commercial, the veh...",business-insider,A cinematographer reveals one thing you probab...,2019-05-17 12:40:00,2019-05-17 13:00:00,218.79,218.58,-0.21,0
168,Silicon Valley billionaire Elon Musk tweeted l...,financial-post,Elon Musk becomes ‘Daddy DotCom’ on Twitter,2019-06-17 09:45:00,2019-06-17 10:05:00,218.07,220.63,2.56,1
73,(Reuters) - A U.S. judge on Friday said Elon M...,reuters,Trial date set for Elon Musk's 'pedo guy' tweet,2019-05-10 15:20:00,2019-05-10 15:40:00,240.38,239.64,-0.74,0
38,Flight attendants reported a significant amoun...,business-insider,38% of flight attendants said in a new survey ...,2019-05-16 11:15:00,2019-05-16 11:35:00,228.77,228.96,0.19,1
128,"AMD\r\nRobinhood, the app popular among young ...",business-insider,Robinhood investors are scooping up AMD after ...,2019-05-30 10:45:00,2019-05-30 11:05:00,187.49,187.44,-0.05,0
